In [ ]:
## Import libraries
import numpy as np
import pandas as pd
import pickle
import time
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import gc
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV

In [ ]:
## Set the display option for wide dataframes
pd.set_option('display.width', 1000)

In [ ]:
## Set variables
N = 10 # experiment repetitions
k = 4  # k for k-fold cross-validation in hyperparameter tuning
seed = 42  # seed for reproducibility

In [ ]:
## Initialize variables to store tuned hyperparameters and results
trials_params, trials_results = [], []

In [ ]:
## Selected clinical dtaa classifier and multi-view thermal image classifier
selected_img_classifier = 'MultiInputCNN'  #'multiInputCNN'
selected_cd_classifier = 'RF'

# Load data

In [ ]:
## Read list of selected patients
f = open('Data/selected_patients.txt', 'r')
patient_IDs = []
for x in f:
  patient_IDs.append(int(x.strip()))

In [ ]:
## Load clinical data
clinical_data = pd.read_csv('Data/clinical_data.csv', header=0, index_col=0, delimiter=';')
clinical_data = clinical_data.loc[patient_IDs]  # Sort the clinical data DataFrame

In [ ]:
## Load images
from PIL import Image

front = np.asarray([np.array(Image.open(os.path.join('Data/Images/Frontal/',str(ID).zfill(4) + '_front.jpg'))) for ID in patient_IDs])
L90 = np.asarray([np.array(Image.open(os.path.join('Data/Images/L90/',str(ID).zfill(4) + '_L90.jpg'))) for ID in patient_IDs])
R90 = np.asarray([np.array(Image.open(os.path.join('Data/Images/R90/',str(ID).zfill(4) + '_R90.jpg'))) for ID in patient_IDs])

In [ ]:
## Generate binary labels
diagnosis = clinical_data['vx-diagnosis']
labels = np.zeros(len(patient_IDs))
labels[diagnosis == 'Sick'] = 1

# Pre-process

## Clinical data

In [ ]:
## Select features
selected_features = ['age_at_visit', 'vx-PH-Eating habits', 'vx-PH-Cancer family?', 'vx-MH-Radiotherapy', 'vx-MH-Use of hormone replacement?',
                     'age_at_menarche', 'age_at_menopause', 'diabetes', 'nodules']
clinical_data = clinical_data[selected_features]

In [ ]:
## Min-max normalization
M = clinical_data.max().values
M[M<1] = 1
m = clinical_data.min().values

clinical_data = (clinical_data-m)/(M-m)

In [ ]:
## Convert to numpy array
clinical_data = np.asarray(clinical_data, dtype=np.float32)

## Thermal images

In [ ]:
## Min-max normalization
M = np.concatenate((front, L90, R90)).max()
m = np.concatenate((front, L90, R90)).min()

front = ((front - m) / (M - m)).astype('float32')
L90 = ((L90 - m) / (M - m)).astype('float32')
R90 = ((R90 - m) / (M - m)).astype('float32')

In [ ]:
## Convert 3-channel to 1-channel images
front = np.expand_dims(front[:,:,:,0], -1)
L90 = np.expand_dims(L90[:,:,:,0], -1)
R90 = np.expand_dims(R90[:,:,:,0], -1)

# Function definitions

In [ ]:
def weighted_error(y_test, y_test_pred):
    WE = 20

    # Number of elements for which y_test > y_test_pred, i.e., y_test = 1 and y_test_pred = 0 (FN)
    fn = np.sum(np.greater(y_test, y_test_pred))

    # Number of elements for which y_test < y_test_pred, i.e., y_test = 0 and y_test_pred = 1 (FP)
    fp = np.sum(np.less(y_test, y_test_pred))

    return fn*WE + fp

In [ ]:
def evaluate_model(preds_raw, ground_truth):

    from sklearn.metrics import confusion_matrix, log_loss, roc_auc_score #, accuracy_score, precision_score, recall_score, f1_score, make_scorer
    
    # BCE loss
    bce = log_loss(ground_truth, preds_raw)

    # Confusion matrix
    TN, FP, FN, TP = confusion_matrix(ground_truth, np.round(preds_raw)).ravel()
    accuracy = (TP + TN) / (TP + TN + FP + FN)  # accuracy = accuracy_score(ground_truth, np.round(preds_raw))
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    #gmean = np.sqrt((TP/(TP+FN))*(TN/(TN+FP)))
    precision = TP / (TP + FP)  # precision = precision_score(ground_truth, np.round(preds_raw))
    F1 = 2 * TP / (2*TP + FP + FN)  # F1 = f1_score(ground_truth, np.round(preds_raw))
    
    # ROC AUC
    auc = roc_auc_score(ground_truth, preds_raw)
    
    # Weighted error    
    we = weighted_error(ground_truth, np.round(preds_raw))
    
    results = {
        'BCELoss':bce,
        'Accuracy':accuracy,
        'TP':TP,
        'FP':FP,
        'TN':TN,
        'FN':FN,
        'Sensitivity':sensitivity,
        'Specificity':specificity,
        #'G-mean':gmean,
        'Precision':precision,
        'Recall':sensitivity,
        'F1':F1,
        'ROC_AUC':auc,
        'WE':we
    }
    return results


# Weighted Voting (WV)

In [ ]:
def weighted_voting(pred_cd, pred_img, weight = 0.5):
    return pred_cd * weight + pred_img * (1 - weight)


from sklearn.metrics import roc_auc_score, accuracy_score
def tune_weight(pred_cd, pred_img, labels, metric=None):
    best_weight = 0
    best_metric = 0
    if metric == "we":
        best_metric = np.inf
            
    for weight_cd in range(0, 10000, 1):
        weight = weight_cd/10000
                
        new_pred = weighted_voting(pred_cd, pred_img, weight)
        
        if metric == "accuracy":
            accuracy = accuracy_score(labels, np.round(new_pred))
            if accuracy > best_metric:
                best_metric = accuracy
                best_weight = weight
        elif metric == "roc_auc":
            roc_auc = roc_auc_score(labels, new_pred)
            if roc_auc > best_metric:
                best_metric = roc_auc
                best_weight = weight
        elif metric == "we":
            we = weighted_error(labels, np.round(new_pred))
            if we < best_metric:
                best_metric = we
                best_weight = weight
        else:
            tune_weight(pred_cd, pred_img, labels, "accuracy")
            tune_weight(pred_cd, pred_img, labels, "roc_auc")
            tune_weight(pred_cd, pred_img, labels, "we")
            break
            
    print(f'Best {metric}: {best_metric} | Best weight: {best_weight}')
    return best_weight

In [ ]:
# Initialize the splitter to split the dataset into 85% train and 15% tests sets N times 
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

In [ ]:
for trial, (train_index, test_index) in enumerate(splitter.split(clinical_data, labels)):
    
    print(f'Trial {trial + 1}'), print()

    ## Split the dataset into train (85%) and test (15%) sets
    crossval_cd = clinical_data[train_index]
    crossval_front = front[train_index]
    crossval_L90 = L90[train_index]
    crossval_R90 = R90[train_index]
    crossval_labels = labels[train_index]
    
    test_cd = clinical_data[test_index]
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Calculate the weight for training to address class imbalance
    n_sick = labels.sum()
    n_healthy = len(labels) - n_sick
    rate_train = n_healthy / n_sick

    ## Load selected clinical data classifier and image classifier
    # Clinical data classifier
    with open('Models/'+selected_cd_classifier+'_'+str(trial+1)+'.pkl','rb') as f:
        cd_classifier = pickle.load(f)

    # Milti-view thermal image classifier
    img_classifier = load_model('Models/'+selected_img_classifier+'_'+str(trial+1)+'.h5')

    ## Get predictions (inputs)
    pred_cd_crossval = cd_classifier.predict_proba(crossval_cd)[:,1]
    pred_img_crossval = img_classifier.predict([crossval_front, crossval_L90, crossval_R90])

    pred_cd_test = cd_classifier.predict_proba(test_cd)[:,1]
    pred_img_test = img_classifier.predict([test_front, test_L90, test_R90])

    
    ## Tune the weight
    ti = time.time()
    weight = tune_weight(pred_cd_crossval, pred_img_crossval, crossval_labels, metric='roc_auc')
    train_time = time.time() - ti
    hours, remainder = divmod(train_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Training took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    ## Store tuned hyperparameters
    trials_params.append({'classifier':'WV', 'trial':trial+1, 'weight':weight})

    ## Predict
    predictions_train = weighted_voting(pred_cd_crossval, pred_img_crossval, weight)
    np.save('Predictions/Ensemble_WV_train_'+str(trial+1)+'.npy',predictions_train)
    predictions_test = weighted_voting(pred_cd_test, pred_img_test, weight)
    np.save('Predictions/Ensemble_WV_test_'+str(trial+1)+'.npy',predictions_test)
        
    ## Evaluate the model
    results_train = evaluate_model(predictions_train, crossval_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()
            
    results_test = evaluate_model(predictions_test, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append(
        {'classifier':'WV',
         'trial':trial+1,
         'parameters':1,
         'trainTime':train_time,
         **{'train_'+k:v for k,v in results_train.items()},
         **{'test_'+k:v for k,v in results_test.items()}
        }
    )

    # Clear cache and TensorFlow session to release GPU memory
    clear_session()
    del img_classifier, cd_classifier, crossval_cd, crossval_front, crossval_L90, crossval_R90, crossval_labels, test_cd, test_front, test_L90, test_R90, test_labels  # Delete the references to objects
    del pred_cd_crossval, pred_img_crossval, pred_cd_test, pred_img_test, predictions_train, predictions_test
    gc.collect()  # Manually invoke garbage collection
    torch.cuda.empty_cache()  # Clear GPU cache

    print(), print(100*'#'), print()

# Decision Tree (DT)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Initialize the splitter to split the dataset into 85% train and 15% tests sets N times 
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

In [ ]:
for trial, (train_index, test_index) in enumerate(splitter.split(clinical_data, labels)):
    
    print(f'Trial {trial + 1}'), print()

    ## Split the dataset into train (85%) and test (15%) sets
    crossval_cd = clinical_data[train_index]
    crossval_front = front[train_index]
    crossval_L90 = L90[train_index]
    crossval_R90 = R90[train_index]
    crossval_labels = labels[train_index]
    
    test_cd = clinical_data[test_index]
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Calculate the weight for training to address class imbalance
    n_sick = labels.sum()
    n_healthy = len(labels) - n_sick
    rate_train = n_healthy / n_sick

    ## Load selected clinical data classifier and image classifier
    # Clinical data classifier
    with open('Models/'+selected_cd_classifier+'_'+str(trial+1)+'.pkl','rb') as f:
        cd_classifier = pickle.load(f)

    # Milti-view thermal image classifier
    img_classifier = load_model('Models/'+selected_img_classifier+'_'+str(trial+1)+'.h5')

    ## Get predictions (inputs)
    pred_cd_crossval = cd_classifier.predict_proba(crossval_cd)[:,1]
    pred_img_crossval = img_classifier.predict([crossval_front, crossval_L90, crossval_R90])
    inputs_crossval = np.concatenate((pred_img_crossval.reshape(-1, 1),pred_cd_crossval.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector

    pred_cd_test = cd_classifier.predict_proba(test_cd)[:,1]
    pred_img_test = img_classifier.predict([test_front, test_L90, test_R90])
    inputs_test = np.concatenate((pred_img_test.reshape(-1, 1),pred_cd_test.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector


    ## Tune hyperparameters with k-fold cross-validation and then train on the entire training set with the tuned hyperparameters
    ti = time.time()
    estimator = DecisionTreeClassifier(class_weight={0: 1, 1: rate_train})
    param_grid = {'ccp_alpha' : np.arange(0, 0.1, 0.01), # Complexity parameter used for Minimal Cost-Complexity Pruning. Default: ccp_alpha = 0.0
                  'criterion': ['gini','entropy'],#'log_loss'], # The function to measure the quality of a split. Default: criterion='gini'
                  'max_depth' : [None, 1, 5, 10, 15], # The maximum depth of the tree. Default: max_depth=None
                  #'max_features': [None, 'sqrt', 'log2'], # The number of features to consider when looking for the best split. Default: max_features=None
                  'max_leaf_nodes': [None, 3, 6, 9], # Grow a tree with max_leaf_nodes in best-first fashion. Default: None
                  'min_samples_leaf': [1, 2, 3, 4], # The minimum number of samples required to be at a leaf node. Default: min_samples_leaf=1
                  'min_samples_split' : [2, 5, 10, 15], # The minimum number of samples required to split an internal node. Default: min_samples_split=2
                  'min_weight_fraction_leaf' : np.arange(0.0, 0.5, 0.05), # The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. Default: min_weight_fraction_leaf=0
                  #'splitter': ['best','random'] # The strategy used to choose the split at each node. Default: splitter='best'
                }
    classifier = GridSearchCV(estimator, param_grid, scoring='roc_auc', cv=k)
    classifier.fit(inputs_crossval, crossval_labels)

    parameters = classifier.best_params_  # Parameter setting that gave the best results on the hold out data
    classifier = classifier.best_estimator_  # Estimator which gave highest score on the left out data

    train_time = time.time() - ti
    hours, remainder = divmod(train_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    ## Store tuned hyperparameters
    trials_params.append({**{'classifier':'DT'}, **{'trial':trial+1}, **parameters})

    ## Save the trained model
    with open('Models/Ensemble_DT_'+str(trial+1)+'.pkl','wb') as f:
        pickle.dump(classifier,f)


    ## Predict
    predictions_train = classifier.predict_proba(inputs_crossval)[:,1]
    np.save('Predictions/Ensemble_DT_train_'+str(trial+1)+'.npy',predictions_train)
    predictions_test = classifier.predict_proba(inputs_test)[:,1]
    np.save('Predictions/Ensemble_DT_test_'+str(trial+1)+'.npy',predictions_test)

    ## Print the number of parameters in the model
    num_params = classifier.tree_.node_count
    print(f'Classifier has {num_params} parameters.'), print()
        
    ## Evaluate the model
    results_train = evaluate_model(predictions_train, crossval_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()
            
    results_test = evaluate_model(predictions_test, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append(
        {'classifier':'DT',
         'trial':trial+1,
         'parameters':num_params,
         'trainTime':train_time,
         **{'train_'+k:v for k,v in results_train.items()},
         **{'test_'+k:v for k,v in results_test.items()}
        }
    )

    # Clear cache and TensorFlow session to release GPU memory
    clear_session()
    del img_classifier, cd_classifier, crossval_cd, crossval_front, crossval_L90, crossval_R90, crossval_labels, test_cd, test_front, test_L90, test_R90, test_labels  # Delete the references to objects
    del pred_cd_crossval, pred_img_crossval, pred_cd_test, pred_img_test, predictions_train, predictions_test
    gc.collect()  # Manually invoke garbage collection
    torch.cuda.empty_cache()  # Clear GPU cache

    print(), print(100*'#'), print()

# Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Initialize the splitter to split the dataset into 85% train and 15% tests sets N times 
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

In [ ]:
for trial, (train_index, test_index) in enumerate(splitter.split(clinical_data, labels)):
    
    print(f'Trial {trial + 1}'), print()

    ## Split the dataset into train (85%) and test (15%) sets
    crossval_cd = clinical_data[train_index]
    crossval_front = front[train_index]
    crossval_L90 = L90[train_index]
    crossval_R90 = R90[train_index]
    crossval_labels = labels[train_index]
    
    test_cd = clinical_data[test_index]
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Calculate the weight for training to address class imbalance
    n_sick = labels.sum()
    n_healthy = len(labels) - n_sick
    rate_train = n_healthy / n_sick

    ## Load selected clinical data classifier and image classifier
    # Clinical data classifier
    with open('Models/'+selected_cd_classifier+'_'+str(trial+1)+'.pkl','rb') as f:
        cd_classifier = pickle.load(f)

    # Milti-view thermal image classifier
    img_classifier = load_model('Models/'+selected_img_classifier+'_'+str(trial+1)+'.h5')

    ## Get predictions (inputs)
    pred_cd_crossval = cd_classifier.predict_proba(crossval_cd)[:,1]
    pred_img_crossval = img_classifier.predict([crossval_front, crossval_L90, crossval_R90])
    inputs_crossval = np.concatenate((pred_img_crossval.reshape(-1, 1),pred_cd_crossval.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector

    pred_cd_test = cd_classifier.predict_proba(test_cd)[:,1]
    pred_img_test = img_classifier.predict([test_front, test_L90, test_R90])
    inputs_test = np.concatenate((pred_img_test.reshape(-1, 1),pred_cd_test.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector

    
    ## Tune hyperparameters with k-fold cross-validation and then train on the entire training set with the tuned hyperparameters
    ti = time.time()
    estimator = SVC(class_weight={0: 1, 1: rate_train}, probability=True)
    param_grid = {'C': [1,10,100,1000], # Regularization parameter. Default: C=1.0
                  'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], # Default: kernel='rbf'
                  'gamma': ['scale', 'auto', 1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001] # Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. Default: gamma='scale'
                 }
    classifier = GridSearchCV(estimator, param_grid, scoring='roc_auc', cv=k)
    classifier.fit(inputs_crossval, crossval_labels)

    parameters = classifier.best_params_  # Parameter setting that gave the best results on the hold out data
    classifier = classifier.best_estimator_  # Estimator which gave highest score on the left out data


    train_time = time.time() - ti
    hours, remainder = divmod(train_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    ## Store tuned hyperparameters
    trials_params.append({**{'classifier':'SVM'}, **{'trial':trial+1}, **parameters})

    ## Save the trained model
    with open('Models/Ensemble_SVM_'+str(trial+1)+'.pkl','wb') as f:
        pickle.dump(classifier,f)


    ## Predict
    predictions_train = classifier.predict_proba(inputs_crossval)[:,1]
    np.save('Predictions/Ensemble_SVM_train_'+str(trial+1)+'.npy',predictions_train)
    predictions_test = classifier.predict_proba(inputs_test)[:,1]
    np.save('Predictions/Ensemble_SVM_test_'+str(trial+1)+'.npy',predictions_test)

    ## Print the number of parameters in the model
    n_support_vectors = len(classifier.support_vectors_)
    n_coefficients = len(classifier.dual_coef_[0])
    num_params = n_support_vectors + n_coefficients
    print(f'Classifier has {num_params} parameters.'), print()
        
    ## Evaluate the model
    results_train = evaluate_model(predictions_train, crossval_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()
            
    results_test = evaluate_model(predictions_test, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append(
        {'classifier':'SVM',
         'trial':trial+1,
         'parameters':num_params,
         'trainTime':train_time,
         **{'train_'+k:v for k,v in results_train.items()},
         **{'test_'+k:v for k,v in results_test.items()}
        }
    )
    
    # Clear cache and TensorFlow session to release GPU memory
    clear_session()
    del img_classifier, cd_classifier, crossval_cd, crossval_front, crossval_L90, crossval_R90, crossval_labels, test_cd, test_front, test_L90, test_R90, test_labels  # Delete the references to objects
    del pred_cd_crossval, pred_img_crossval, pred_cd_test, pred_img_test, predictions_train, predictions_test
    gc.collect()  # Manually invoke garbage collection
    torch.cuda.empty_cache()  # Clear GPU cache

    print(), print(100*'#'), print()

# Neural Network (NN)

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
# Initialize the splitter to split the dataset into 85% train and 15% tests sets N times 
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

In [ ]:
for trial, (train_index, test_index) in enumerate(splitter.split(clinical_data, labels)):
    
    print(f'Trial {trial + 1}'), print()

    ## Split the dataset into train (85%) and test (15%) sets
    crossval_cd = clinical_data[train_index]
    crossval_front = front[train_index]
    crossval_L90 = L90[train_index]
    crossval_R90 = R90[train_index]
    crossval_labels = labels[train_index]
    
    test_cd = clinical_data[test_index]
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Calculate the weight for training to address class imbalance
    n_sick = labels.sum()
    n_healthy = len(labels) - n_sick
    rate_train = n_healthy / n_sick

    ## Load selected clinical data classifier and image classifier
    # Clinical data classifier
    with open('Models/'+selected_cd_classifier+'_'+str(trial+1)+'.pkl','rb') as f:
        cd_classifier = pickle.load(f)

    # Milti-view thermal image classifier
    img_classifier = load_model('Models/'+selected_img_classifier+'_'+str(trial+1)+'.h5')

    ## Get predictions (inputs)
    pred_cd_crossval = cd_classifier.predict_proba(crossval_cd)[:,1]
    pred_img_crossval = img_classifier.predict([crossval_front, crossval_L90, crossval_R90])
    inputs_crossval = np.concatenate((pred_img_crossval.reshape(-1, 1),pred_cd_crossval.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector

    pred_cd_test = cd_classifier.predict_proba(test_cd)[:,1]
    pred_img_test = img_classifier.predict([test_front, test_L90, test_R90])
    inputs_test = np.concatenate((pred_img_test.reshape(-1, 1),pred_cd_test.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector


    ## Tune hyperparameters with k-fold cross-validation and then train on the entire training set with the tuned hyperparameters
    ti = time.time()
    estimator = Perceptron(class_weight={0: 1, 1: rate_train})
    param_grid = {'penalty': [None, 'l2', 'l1', 'elasticnet'], # The penalty (aka regularization term). Default: penalty=None
                  'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], # Constant that multiplies the regularization term if regularization is used. Default: alpha=0.0001
                  'fit_intercept': [True, False], # Whether the intercept should be estimated or not. Default: fit_intercept=True
                  'max_iter': [500, 1000, 1500, 2000], # The maximum number of passes over the training data (aka epochs). Default: max_iter=1000
                  'tol': [None, 1e-2, 1e-3, 1e-4, 1e-5], # The stopping criterion. Default: tol=1e-3
                  #'shuffle': [True, False], # Whether or not the training data should be shuffled after each epoch. Default: shuffle=True
                  'eta0': [10, 1, 0.1, 0.01, 0.001], # Constant by which the updates are multiplied. Default: eta0=1
                  'validation_fraction': [0.01, 0.1, 0.2, 0.3] # The proportion of training data to set aside as validation set for early stopping. Default: validation_fraction=0.1
                }
    classifier = GridSearchCV(estimator, param_grid, scoring='roc_auc', cv=k)
    classifier.fit(inputs_crossval, crossval_labels)

    parameters = classifier.best_params_  # Parameter setting that gave the best results on the hold out data
    classifier = classifier.best_estimator_  # Estimator which gave highest score on the left out data

    train_time = time.time() - ti
    hours, remainder = divmod(train_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    ## Store tuned hyperparameters
    trials_params.append({**{'classifier':'NN'}, **{'trial':trial+1}, **parameters})

    ## Save the trained model
    with open('Models/Ensemble_NN_'+str(trial+1)+'.pkl','wb') as f:
        pickle.dump(classifier,f)


    ## Predict
    predictions_train = classifier.predict(inputs_crossval)
    np.save('Predictions/Ensemble_NN_train_'+str(trial+1)+'.npy',predictions_train)
    predictions_test = classifier.predict(inputs_test)
    np.save('Predictions/Ensemble_NN_test_'+str(trial+1)+'.npy',predictions_test)

    ## Print the number of parameters in the model
    num_params = classifier.coef_.size + classifier.intercept_.size
    print(f'Classifier has {num_params} parameters.'), print()
        
    ## Evaluate the model
    results_train = evaluate_model(predictions_train, crossval_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()
            
    results_test = evaluate_model(predictions_test, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append(
        {'classifier':'NN',
         'trial':trial+1,
         'parameters':num_params,
         'trainTime':train_time,
         **{'train_'+k:v for k,v in results_train.items()},
         **{'test_'+k:v for k,v in results_test.items()}
        }
    )

    # Clear cache and TensorFlow session to release GPU memory
    clear_session()
    del img_classifier, cd_classifier, crossval_cd, crossval_front, crossval_L90, crossval_R90, crossval_labels, test_cd, test_front, test_L90, test_R90, test_labels  # Delete the references to objects
    del pred_cd_crossval, pred_img_crossval, pred_cd_test, pred_img_test, predictions_train, predictions_test
    gc.collect()  # Manually invoke garbage collection
    torch.cuda.empty_cache()  # Clear GPU cache

    print(), print(100*'#'), print()

# Stochastic Gradient Descent (SGD)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
# Initialize the splitter to split the dataset into 85% train and 15% tests sets N times 
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

In [ ]:
for trial, (train_index, test_index) in enumerate(splitter.split(clinical_data, labels)):
    
    print(f'Trial {trial + 1}'), print()

    ## Split the dataset into train (85%) and test (15%) sets
    crossval_cd = clinical_data[train_index]
    crossval_front = front[train_index]
    crossval_L90 = L90[train_index]
    crossval_R90 = R90[train_index]
    crossval_labels = labels[train_index]
    
    test_cd = clinical_data[test_index]
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Calculate the weight for training to address class imbalance
    n_sick = labels.sum()
    n_healthy = len(labels) - n_sick
    rate_train = n_healthy / n_sick

    ## Load selected clinical data classifier and image classifier
    # Clinical data classifier
    with open('Models/'+selected_cd_classifier+'_'+str(trial+1)+'.pkl','rb') as f:
        cd_classifier = pickle.load(f)

    # Milti-view thermal image classifier
    img_classifier = load_model('Models/'+selected_img_classifier+'_'+str(trial+1)+'.h5')

    ## Get predictions (inputs)
    pred_cd_crossval = cd_classifier.predict_proba(crossval_cd)[:,1]
    pred_img_crossval = img_classifier.predict([crossval_front, crossval_L90, crossval_R90])
    inputs_crossval = np.concatenate((pred_img_crossval.reshape(-1, 1),pred_cd_crossval.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector

    pred_cd_test = cd_classifier.predict_proba(test_cd)[:,1]
    pred_img_test = img_classifier.predict([test_front, test_L90, test_R90])
    inputs_test = np.concatenate((pred_img_test.reshape(-1, 1),pred_cd_test.reshape(-1, 1)), axis=1)  # Concatenate predictions to generate the input vector


    ## Tune hyperparameters with k-fold cross-validation and then train on the entire training set with the tuned hyperparameters
    ti = time.time()
    estimator = SGDClassifier(class_weight={0: 1, 1: rate_train})
    param_grid = {'loss': ['log_loss', 'modified_huber'], # The loss function to be used. Default: loss='hinge'. Options: ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', ‘squared_error’, ‘huber’, ‘epsilon_insensitive’, ‘squared_epsilon_insensitive’]
                  'penalty': [None, 'l2', 'l1', 'elasticnet'], # The penalty (aka regularization term) to be used. Default: penalty='l2'
                  'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], # Constant that multiplies the regularization term. Default: alpha=0.0001
                  'l1_ratio': [0.05, 0.15, 0.3, 0.5, 0.7, 0.9],  # The Elastic Net mixing parameter, only used if penalty is 'elasticnet'. Default: l1_ratio=0.15
                  'max_iter': [500, 1000, 1500, 2000], # The maximum number of passes over the training data (aka epochs). Default: max_iter=1000
                  'tol': [None, 1e-2, 1e-3, 1e-4, 1e-5], # The stopping criterion. Default: tol=1e-3
                  'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'], # The learning rate schedule. Default: learning_rate='optimal'
                  'eta0':  [0.001, 0.01, 0.1, 1, 10]  # The initial learning rate for the ‘constant’, ‘invscaling’ or ‘adaptive’ schedules. Default: eta0=0.0
                }
    classifier = GridSearchCV(estimator, param_grid, scoring='roc_auc', cv=k)
    classifier.fit(inputs_crossval, crossval_labels)

    parameters = classifier.best_params_  # Parameter setting that gave the best results on the hold out data
    classifier = classifier.best_estimator_  # Estimator which gave highest score on the left out data

    train_time = time.time() - ti
    hours, remainder = divmod(train_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    ## Store tuned hyperparameters
    trials_params.append({**{'classifier':'SGD'}, **{'trial':trial+1}, **parameters})

    ## Save the trained model
    with open('Models/Ensemble_SGD_'+str(trial+1)+'.pkl','wb') as f:
        pickle.dump(classifier,f)


    ## Predict
    predictions_train = classifier.predict_proba(inputs_crossval)[:,1]
    np.save('Predictions/Ensemble_SGD_train_'+str(trial+1)+'.npy',predictions_train)
    predictions_test = classifier.predict_proba(inputs_test)[:,1]
    np.save('Predictions/Ensemble_SGD_test_'+str(trial+1)+'.npy',predictions_test)

    ## Print the number of parameters in the model
    num_params = classifier.coef_.size + classifier.intercept_.size
    print(f'Classifier has {num_params} parameters.'), print()
        
    ## Evaluate the model
    results_train = evaluate_model(predictions_train, crossval_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()
            
    results_test = evaluate_model(predictions_test, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append(
        {'classifier':'SGD',
         'trial':trial+1,
         'parameters':num_params,
         'trainTime':train_time,
         **{'train_'+k:v for k,v in results_train.items()},
         **{'test_'+k:v for k,v in results_test.items()}
        }
    )

    # Clear cache and TensorFlow session to release GPU memory
    clear_session()
    del img_classifier, cd_classifier, crossval_cd, crossval_front, crossval_L90, crossval_R90, crossval_labels, test_cd, test_front, test_L90, test_R90, test_labels  # Delete the references to objects
    del pred_cd_crossval, pred_img_crossval, pred_cd_test, pred_img_test, predictions_train, predictions_test
    gc.collect()  # Manually invoke garbage collection
    torch.cuda.empty_cache()  # Clear GPU cache

    print(), print(100*'#'), print()

In [ ]:
## Store tuned hyperparameters and results
pd.DataFrame(trials_params).to_csv('Ensemble_Parameters_'+str(N)+'trials.csv')
pd.DataFrame(trials_results).round(decimals=5).to_csv('Ensemble_Results_'+str(N)+'trials.csv')

# Compare models

In [ ]:
trials_results = pd.DataFrame(trials_results).fillna(1e-9)

In [ ]:
## Print statistics
models = trials_results.classifier.unique()
metrics = [c for c in trials_results.columns if 'test_' in c and c not in ['test_TP','test_FP','test_TN','test_FN']]
    
statistics = pd.DataFrame(index=models, columns=[item for sublist in [[metric+'_mean', metric+'_std'] for metric in metrics] for item in sublist])
    
for metric in metrics:
    mn, st = metric+'_mean', metric+'_std'
    for model in models:
        results = trials_results[trials_results['classifier']==model][metric].values
        statistics.at[model,mn] = results.mean()
        statistics.at[model,st] = results.std()

statistics

In [ ]:
for metric in [m for m in metrics if 'test' in m]:
    mn, st = metric+'_mean', metric+'_std'
    if 'Loss' in metric or 'WE' in metric:
        model_best = pd.to_numeric(statistics[metric+'_mean']).idxmin()
        print(f'Model with lowest {metric} is {model_best} with value {statistics.loc[model_best,mn]} and standard deviation {statistics.loc[model_best,st]}')
    else:
        model_best = pd.to_numeric(statistics[metric+'_mean']).idxmax()
        print(f'Model with highest {metric} is {model_best} with value {statistics.loc[model_best,mn]} and standard deviation {statistics.loc[model_best,st]}')

In [ ]:
## Print mean and std metrics for each model
for classifier_type in trials_results.classifier.unique():
    print(f'Classifier: {classifier_type}')
    results = trials_results[trials_results['classifier'] == classifier_type]

    # Number of parameters
    parameters = results['parameters'].values
    print(f'Mean number of parameters: {parameters.mean()} [{parameters.min()}, {parameters.max()}], std {parameters.std()}')

    # training time
    trainTime = results['trainTime'].values
    hours, remainder = divmod(trainTime.mean(), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Mean training time: {hours} hours, {minutes} minutes, and {seconds} seconds, (std {trainTime.std()} sec)')
    print()
    
    # TRAIN results
    metrics = ['BCELoss','Accuracy','Sensitivity','Specificity','ROC_AUC','Precision','F1','WE']
    for metric in metrics:
        values = results['train_' + metric].values
        print(f'Mean train {metric}: {values.mean()}, std {values.std()}')
    print()

    # TEST results
    for metric in metrics:
        values = results['test_' + metric].values
        print(f'Mean test {metric}: {values.mean()}, std {values.std()}')
    print()

    print('-'*120), print()

In [ ]:
## Show boxplots
compared_metrics = ['test_BCELoss','test_Accuracy','test_F1','test_ROC_AUC','test_WE']  #[c for c in cd_trials_results.columns if 'test_' in c and c not in ['test_TP','test_FP','test_TN','test_FN','test_WE','test_Loss']]
M = len(compared_metrics)
R = M//2 + int(M % 2 > 0)
plt.figure(figsize=(5*R, 20))
for i,metric in enumerate(compared_metrics):
    plt.subplot(R,2,i+1)
    sns.boxplot(x='classifier', y=metric, data=trials_results, palette='Set2')
    plt.xticks(rotation=25, ha='right')  # Rotate labels 25 degrees
    plt.grid(axis='y')
    plt.xlabel('')
    if 'Loss' in metric or 'WE' in metric:
        plt.ylim([0,trials_results[metric].values.max()+0.05*trials_results[metric].values.max()])
    else:
        plt.ylim([0,1])
    plt.title(metric, fontsize=16)
    #plt.tight_layout()
    
plt.savefig('Ensemble_Boxplots_allModels.png')
plt.subplots_adjust(hspace=0.3)  # Increase hspace for more vertical spacing
plt.show()

In [ ]:
## Statistical model comparison
from scipy.stats import shapiro
from scipy.stats import probplot
from statsmodels.multivariate.manova import MANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd
    
models = trials_results.classifier.unique()
compared_metrics = ['test_BCELoss','test_Accuracy','test_F1','test_ROC_AUC','test_WE']
    
## 1. Select independent metrics
#These should be independent. Use the correlation to find dependent variables to remove, if necessary 
print('Check that the metrics are independent:')
print(trials_results[compared_metrics].corr()), print()  # Check for multicollinearity using the correlation matrix

var = trials_results[compared_metrics].var().to_numpy()
compared_metrics = [metric for metric,v in zip(compared_metrics,var) if not v == 0]  # Remove metrics with zero variance across repetitions

## 2. Check for Normality
normality = np.ones(len(compared_metrics))
L = len(models)//5 + (len(models) % 5 > 0)
for i,metric in enumerate(compared_metrics): 
    print(metric)
    ## Check for normality
    plt.figure(figsize=(20,2*L))
    for j,model in enumerate(models):
        # Shapiro-Wilk test
        stat, p_value = shapiro(trials_results[trials_results['classifier']==model][metric])
        print(f'{model}: Shapiro-Wilk p-value={p_value}')
        # If p-value < 0.05, reject normality (non-normal distribution)
        normality[i] *= int(p_value > 0.05)

        # Generate Q-Q plots to visually inspect normality 
        plt.subplot(L,5,j+1)
        probplot(trials_results[trials_results['classifier']==model][metric], dist='norm', plot=plt)
        plt.title(model)
    plt.show()
    print(f'\nNormality test for {metric}: {bool(normality[i])}')
    print(100*'-')
print(), print()
    
    
## 3. Statistical test: MANOVA
dependent_variables = ' + '.join(compared_metrics)
formula = f'{dependent_variables} ~ classifier'
manova = MANOVA.from_formula(formula, data=trials_results)
manova_test = manova.mv_test()
print(manova_test)

## 4. Post-hoc test: Tukey's HSD
if manova_test.results['classifier']['stat']['Pr > F']['Pillai\'s trace'] < 0.05:
    for metric in compared_metrics:
        print(metric)
        tukey = pairwise_tukeyhsd(trials_results[metric],   # Metric data
                                          trials_results['classifier'],   # Grouping variable (model)
                                          alpha=0.05)   # Significance level
        print(tukey)
else:
    print(f"MANOVA is not significant, no need to apply Tukey's test.")

# Selected model

In [ ]:
selected_model = 'WV'
results_selected = trials_results[trials_results['classifier']==selected_model]

In [ ]:
## Show boxplots
df_long = pd.melt(results_selected[compared_metrics], value_vars=compared_metrics, var_name='metric', value_name='value')
plt.figure(figsize=(8, 5))
sns.boxplot(y='value',x='metric',data=df_long, palette='Set2', fliersize=5)
new_labels = [l.replace('test_','') for l in compared_metrics]
plt.xticks(ticks=range(len(new_labels)), labels=new_labels, fontsize=18, rotation=25, ha='right')
plt.yticks(fontsize=18)
plt.grid(axis='y')
plt.xlabel('', fontsize=18)
plt.ylabel('', fontsize=18)
plt.ylim([-0.05,1.05])
plt.savefig('Ensemble_Boxplot_'+selected_model+'.png')
plt.show()